# 1. INTRODUCCIÓ

La base de datos en la que nos centraremos en este proyecto, se basa en dos archivos en los cuales hay un listado de artistas. Cada artista tiene asignada una cancion y aparte nos dan diferentes caracteristicas como pueden ser la popularidad, la energia o el tempo, entre otros. 
Nuestro trabajo va a consistir en hacer una prediccion usando la regressión logistica. 

# 2. DEPENDENCIAS
**pandas:** Todas las operaciones de vectores de datos y matrices.                                           
**sklearn:** Graficando visualmente los resultados obtenidos.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression as lrr
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn import svm, datasets,preprocessing
from sklearn.model_selection import train_test_split



from sklearn.naive_bayes import GaussianNB as gnb
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score as cvs, GridSearchCV as gscv, StratifiedKFold as skf
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler as ss, MinMaxScaler as mms, RobustScaler as rs

# 3. EXPLORACION BASE DATOS
Antes de empezar a trabajar sobre el proyecto debemos analizar nuestros datos y familiarizarnos con ellos.Todo esto nos servirá para después tomar las decisiones correctas.
La base de datos se basa en 3 archivos:                                                                                    
El primer archivo es el de train.csv, que es el conjunto de entrenamiento:

In [2]:
df = pd.read_csv("C:/Users/Usuario/Desktop/KAGGLE/train.csv")

print ( 'Total de artistas:' , df.shape[ 0 ]) 
print ( 'Numero de Atributos:' , df.shape[ 1 ])
df.head()


Total de artistas: 17996
Numero de Atributos: 17


,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,NaN,0.0849,0.8990,134.071,234596.0,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4,10
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,NaN,0.1220,0.5690,107.033,173968.0,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4,10


El siguiente archivo es el test.csv, es el conjunto de test.

In [3]:
dte = pd.read_csv("C:/Users/Usuario/Desktop/KAGGLE/test.csv")
dte

print ( 'Total de artistas:' , dte.shape[ 0 ]) 
print ( 'Numero de Atributos:' , dte.shape[ 1 ])

dte.head()

Total de artistas: 7713
Numero de Atributos: 16


,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature
0,David Bowie,Space Oddity - 2015 Remaster,73.0,0.310,0.403,NaN,-13.664,1,0.0326,0.07260,0.000093,0.1390,0.466,134.480,318027.000000,4
1,Crimson Sun,Essence of Creation,34.0,0.511,0.955,1.0,-5.059,1,0.1290,0.00040,0.000009,0.2630,0.291,151.937,220413.000000,4
2,P!nk,Raise Your Glass,78.0,0.700,0.709,7.0,-5.006,1,0.0839,0.00480,NaN,0.0289,0.625,122.019,202960.000000,4
3,Shawn Mendes,Wonder,80.0,0.333,0.637,1.0,-4.904,0,0.0581,0.13100,0.000018,0.1490,0.132,139.898,172693.000000,4
4,Backstreet Boys,Helpless When She Smiles - Radio Version,48.0,0.393,0.849,11.0,-4.114,1,0.0459,0.00421,NaN,0.1620,0.222,74.028,4.093117,4


Por úlimo tenemos el archivo submission:

In [4]:
s = pd.read_csv("C:/Users/Usuario/Desktop/KAGGLE/submission.csv")
s.head()

,Acoustic/Folk_0,Alt_Music_1,Blues_2,Bollywood_3,Country_4,HipHop_5,Indie Alt_6,Instrumental_7,Metal_8,Pop_9,Rock_10
0,1,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0


Después de analizar a fondo toda nuestra base de datos, he decidido que el atributo target, el que utilizaremos para clasificar serà el atributo 'Clase', que nos indica a que clase musical pertenece cada artista.

## Tratamiento de los valores nulos

Para el tratamiento de valors nulos en nuestra base de datos, lo que haremos es intercanviarlos por otro valores. Lo primero de todo es detectar la cantidad de valores nulos que hay y donde se encuentran en nuestro dataset.

In [5]:
###Train
null_feats = pd.DataFrame(df.isnull().sum(), columns = ['nans']).sort_values('nans', ascending = False)
null_feats['nans %'] = np.round(df.isnull().sum() / df.shape[0], 2)
print(null_feats.head())

###Test
null_feats_test = pd.DataFrame(dte.isnull().sum(), columns = ['nans']).sort_values('nans', ascending = False)
null_feats_test['nans %'] = np.round(dte.isnull().sum() / dte.shape[0], 2)
null_feats_test.head()


                  nans  nans %
instrumentalness  4377    0.24
key               2014    0.11
Popularity         428    0.02
Artist Name          0    0.00
time_signature       0    0.00


,nans,nans %
instrumentalness,1909,0.25
key,808,0.10
Popularity,227,0.03
Artist Name,0,0.00
Track Name,0,0.00


In [6]:
## TRAIN
nulls = null_feats.index[:3]
print(df[nulls].describe())

## TEST
nulls_test = null_feats_test.index[:3]
dte[nulls_test].describe()

       instrumentalness           key    Popularity
count      13619.000000  15982.000000  17568.000000
mean           0.177562      5.952447     44.512124
std            0.304048      3.196854     17.426928
min            0.000001      1.000000      1.000000
25%            0.000089      3.000000     33.000000
50%            0.003910      6.000000     44.000000
75%            0.200000      9.000000     56.000000
max            0.996000     11.000000    100.000000


,instrumentalness,key,Popularity
count,5804.000000,6905.000000,7486.000000
mean,0.170052,5.924692,44.968074
std,0.298011,3.238735,17.401707
min,0.000001,1.000000,1.000000
25%,0.000092,3.000000,33.000000
50%,0.004010,6.000000,44.000000
75%,0.165250,9.000000,57.000000
max,0.987000,11.000000,100.000000


Una vez tengo detectados todos los nulos y en que columnas se encuentran, procedo a cambiar los valores nulos por la media correspondiente a cada una de las columnas. Por ejemplo hemos calculado que la media en la columna 'instruentalness' es de 0.177562, entonces todos los valores nulos de esta columna pasaran a tener el valor 0.177562

In [7]:
## TRAIN
df[nulls] = df[nulls].fillna(df[nulls].mean())
print(df.head())

## TEST
dte[nulls_test] = dte[nulls_test].fillna(dte[nulls_test].mean())
dte.head()

             Artist Name                             Track Name  Popularity  \
0             Bruno Mars  That's What I Like (feat. Gucci Mane)        60.0   
1                 Boston                           Hitch a Ride        54.0   
2          The Raincoats                     No Side to Fall In        35.0   
3                   Deno             Lingo (feat. J.I & Chunkz)        66.0   
4  Red Hot Chili Peppers      Nobody Weird Like Me - Remastered        53.0   

   danceability  energy   key  loudness  mode  speechiness  acousticness  \
0         0.854   0.564   1.0    -4.964     1       0.0485      0.017100   
1         0.382   0.814   3.0    -7.230     1       0.0406      0.001100   
2         0.434   0.614   6.0    -8.334     1       0.0525      0.486000   
3         0.853   0.597  10.0    -6.528     0       0.0555      0.021200   
4         0.167   0.975   2.0    -4.279     1       0.2160      0.000169   

   instrumentalness  liveness  valence    tempo  duration_in min/ms 

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature
0,David Bowie,Space Oddity - 2015 Remaster,73.0,0.310,0.403,5.924692,-13.664,1,0.0326,0.07260,0.000093,0.1390,0.466,134.480,318027.000000,4
1,Crimson Sun,Essence of Creation,34.0,0.511,0.955,1.000000,-5.059,1,0.1290,0.00040,0.000009,0.2630,0.291,151.937,220413.000000,4
2,P!nk,Raise Your Glass,78.0,0.700,0.709,7.000000,-5.006,1,0.0839,0.00480,0.170052,0.0289,0.625,122.019,202960.000000,4
3,Shawn Mendes,Wonder,80.0,0.333,0.637,1.000000,-4.904,0,0.0581,0.13100,0.000018,0.1490,0.132,139.898,172693.000000,4
4,Backstreet Boys,Helpless When She Smiles - Radio Version,48.0,0.393,0.849,11.000000,-4.114,1,0.0459,0.00421,0.170052,0.1620,0.222,74.028,4.093117,4


# 4. PREDICCIÓN:  REGRESIÓN LOGÍSTICA

In [8]:
del df["Track Name"]

# Typecasting
df["Artist Name"] = df["Artist Name"].astype(str)

# Initializing Encoder
number = preprocessing.LabelEncoder()

# Encoding
df["Artist Name"] = number.fit_transform(df["Artist Name"])

df.head()


,Artist Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
0,1182,60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,0.177562,0.0849,0.8990,134.071,234596.0,4,5
1,1092,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4,10
2,7899,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4,6
3,1987,66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,0.177562,0.1220,0.5690,107.033,173968.0,4,5
4,6283,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4,10


In [9]:
X = df.drop(["Class"], axis = 1).values

y = df["Class"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.2)

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier

model = OneVsOneClassifier(LogisticRegression())
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.5011111111111111


# 5. MODELOS DE CLASIFICACIÓN

Con el uso de la libreria pycaret, veremos cual de todos los modelos de classificación es el más óptimo de todos.

In [12]:
from pycaret.classification import *

In [14]:
dataset = pd.read_csv("C:/Users/Usuario/Desktop/KAGGLE/train.csv")
dataset.head()

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,NaN,0.0849,0.8990,134.071,234596.0,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4,10
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,NaN,0.1220,0.5690,107.033,173968.0,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4,10


In [15]:
data = dataset.sample(frac=0.95, random_state=786).reset_index(drop=True)
data_unseen = dataset.drop(data.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (17096, 17)
Unseen Data For Predictions: (900, 17)


In [18]:
classifier = setup(data = data, target = 'Class',ignore_features = ['Artist Name','Track Name'] ,session_id=123)

,Description,Value
0,session_id,123
1,Target,Class
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(17096, 17)"
5,Missing Values,True
6,Numeric Features,11
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [19]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.5381,0.8486,0.5720,0.5144,0.5146,0.4459,0.4520,7.4080
lightgbm,Light Gradient Boosting Machine,0.5188,0.8409,0.5654,0.5024,0.5053,0.4257,0.4288,0.5900
rf,Random Forest Classifier,0.5102,0.8039,0.5580,0.4933,0.4943,0.4136,0.4177,0.5000
et,Extra Trees Classifier,0.4941,0.7551,0.5271,0.4799,0.4781,0.3928,0.3974,0.3840
lda,Linear Discriminant Analysis,0.4810,0.8050,0.5209,0.4416,0.4490,0.3777,0.3849,0.0440
ridge,Ridge Classifier,0.4373,0.0000,0.3429,0.4188,0.3799,0.2981,0.3214,0.0220
dt,Decision Tree Classifier,0.3679,0.6304,0.4478,0.3787,0.3723,0.2635,0.2638,0.0570
nb,Naive Bayes,0.3365,0.7061,0.3198,0.2966,0.2442,0.1512,0.1867,0.0250
ada,Ada Boost Classifier,0.2975,0.7011,0.3460,0.3155,0.2110,0.1985,0.2302,0.2300
lr,Logistic Regression,0.2920,0.6609,0.1794,0.1412,0.1509,0.0782,0.1155,0.8500


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=123, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

# 6. CONCLUSIONES Y TRABAJOS FUTUROS 

Como conclusion, he podido observar como el modelo más optimo para clasificar nuestra base de datos ha sido Gradient Boosting Classifier con un Accuracy de 0.5381.A parte de tener el accuracy más óptimo, también da mejores resultados en precision o en recall. A parte de este modelo, también podemos ver otros como el Random Forest que seria el tercer clasificador más eficiente. 
Como trabajos futuros podemos implementar diferentes modelos de clasificacion, asi como otra formas de predicción. 